# Befolkningsutveckling i Stockholms län

## Inledning
Detta skript hämtar befolkningsstatistik med API från SCB för Stockholms läns kommuner och visualiserar detta i en interaktiv Dash app.

## Hämta data från SCB
För att hämta statistik från SCB med API används biblioteket Pyscbwrapper.

Läs mer om Pyscbwrapper här: https://github.com/kirajcg/pyscbwrapper/blob/master/pyscbwrapper.ipynb

In [1]:
#Importera bibliotek
from pyscbwrapper import SCB
import pandas as pd

In [2]:
#Initiera SCB
scb = SCB('sv')

#Välj tabell
scb.go_down('BE', 'BE0101', 'BE0101G', 'ManadBefStatRegion')

In [3]:
#Ladda hem data

# Skapa en fråga att skicka till databasen
scb.set_query(region = ['Stockholms län',
                        'Upplands Väsby',
                        'Vallentuna',
                        'Österåker',
                        'Värmdö',
                        'Järfälla',
                        'Ekerö',
                        'Huddinge',
                        'Botkyrka',
                        'Salem',
                        'Haninge',
                        'Tyresö',
                        'Upplands-Bro',
                        'Nykvarn',
                        'Täby',
                        'Danderyd',
                        'Sollentuna',
                        'Stockholm',
                        'Södertälje',
                        'Nacka',
                        'Sundbyberg',
                        'Solna',
                        'Lidingö',
                        'Vaxholm',
                        'Norrtälje',
                        'Sigtuna',
                        'Nynäshamn'],
              förändringar = ['folkmängd'],
              kön = ['totalt'])

# Inspektera fråga
scb.get_query()

# Hämta metadata och extrahera data
scb_metadata = scb.get_data()
scb_data = scb_metadata['data']

# Inspektera endast de främsta raderna av data i JSON format
num_rows = 3

# Inspektera de tre första raderna
for row in scb_data[:num_rows]:
    print(row)

{'key': ['01', '100', '1+2', '2000M01'], 'values': ['1804587']}
{'key': ['01', '100', '1+2', '2000M02'], 'values': ['1806089']}
{'key': ['01', '100', '1+2', '2000M03'], 'values': ['1808101']}


In [4]:
# Konvertera en lista av dictionaries till en dictionary med dictionaries
scb_dict = {tuple(d['key']): d['values'][0] for d in scb_data}

# Skapa en DataFrame av dictionary
df = pd.DataFrame.from_dict(scb_dict, orient='index', columns=['value'])

# Sätt ett namn för index
df.index.set_names('keys', inplace=True)
df.reset_index(inplace=True)
df[['region', 'förändringar', 'kön', 'månad']] = df['keys'].apply(pd.Series)
df.drop('keys', axis=1, inplace=True)
df.rename(columns={'value':'befolkning'}, inplace=True)

In [5]:
#Lägg till kommunnamn som kolumn

# Definiera mappning
kommun_mapping = {
    '01': 'Stockholms län',
    '0114': 'Upplands Väsby',
    '0115': 'Vallentuna',
    '0117': 'Österåker',
    '0120': 'Värmdö',
    '0123': 'Järfälla',
    '0125': 'Ekerö',
    '0126': 'Huddinge',
    '0127': 'Botkyrka',
    '0128': 'Salem',
    '0136': 'Haninge',
    '0138': 'Tyresö',
    '0139': 'Upplands-Bro',
    '0140': 'Nykvarn',
    '0160': 'Täby',
    '0162': 'Danderyd',
    '0163': 'Sollentuna',
    '0180': 'Stockholm',
    '0181': 'Södertälje',
    '0182': 'Nacka',
    '0183': 'Sundbyberg',
    '0184': 'Solna',
    '0186': 'Lidingö',
    '0187': 'Vaxholm',
    '0188': 'Norrtälje',
    '0191': 'Sigtuna',
    '0192': 'Nynäshamn'
}

# Skapa 'kommun' variabel baserad på 'region'
df['kommun'] = df['region'].str[:4].map(kommun_mapping)
df = df[df['förändringar'] == '100']
df.drop(['kön', 'förändringar', 'region'], axis=1, inplace=True)

#Transformera 'befolkning' till integer
df['befolkning'] = df['befolkning'].astype(int)

#Transformera 'månad' till datum
df['månad'] = df['månad'].str.replace("M", "-")
df['månad'] = df['månad']+'-01'
df['månad'] = pd.to_datetime(df['månad'])

#Filtrera df till datum från 2015-01-01
df = df[df['månad'] >= '2015-01-01']

df.head()

,befolkning,månad,kommun
180,2200237,2015-01-01,Stockholms län
181,2202445,2015-02-01,Stockholms län
182,2205105,2015-03-01,Stockholms län
183,2207908,2015-04-01,Stockholms län
184,2210572,2015-05-01,Stockholms län


## Skapa Dash app och visualisera

### Skapa Dash App

In [6]:
#Importera Plotly och Dash
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import plotly.io as pio

In [7]:
#Skapa drop-down lista
kommuner = [{'label': kommun, 'value': kommun} for kommun in df['kommun'].unique()]

In [32]:
#Bygg app
app = Dash(__name__)

# Sätt Verdana som globalt typsnitt
font_family_verdana = 'Verdana'

app.layout = html.Div([
    html.H2('Stockholms läns befolkningsutveckling', style={'font-family': font_family_verdana}),
    html.Div(
    children=[
        html.P('Välj kommun:', style={'font-family': font_family_verdana}),
        html.Div(
            children=[
                dcc.Dropdown(id='kommun_dd',
                    # Sätt val som ska finnas i drop-down filter
                    options=kommuner,
                    value="Stockholms län",
                    clearable=False,
                    style={'font-family': font_family_verdana}
                    ),
                ], style={'width': '50%', 'display': 'inline-block'}),
        html.Div(
            children=[
                # Lägg till diagramkomponent
                dcc.Graph(id='Graph'),
                html.P('Källa: SCB', style={'font-family': font_family_verdana}),
                html.P('En Dash app av Emanuel Raptis', style={'font-family': font_family_verdana, 'font-style': 'italic'})]),
            ]),
    ],
)

@app.callback(
    # Sätt input and output för callback och länka dropdown till diagrammet
    Output(component_id='Graph', component_property='figure'),
    Input(component_id='kommun_dd', component_property='value')
)

#Skapa interaktivt plotly diagram
def update_plot(input_kommun):
    df_befolkning = df.copy(deep=True) #Skapa kopia av dataframe
    if input_kommun:
        kommun = input_kommun
        df_befolkning = df_befolkning[df_befolkning['kommun'] == input_kommun]
    
    #Skapa plotly diagram    
    fig_folkmangd = px.line(df_befolkning,
                            x="månad", y="befolkning",
                            hover_name="kommun",
                            markers=True,
                            title=f'Folkmängd per månad i {input_kommun}')  #Inkludera vald kommun i rubrik

    #Lägg till knappar för att styra x-axeln
    fig_folkmangd.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=list([
                dict(count=2, label="Innevarande år", step="year", stepmode="todate"),
                dict(count=3, label="3 år", step="year", stepmode="backward"),
                dict(count=5, label="5 år", step="year", stepmode="backward"),
                dict(step="all", label="Hela perioden")
            ])
        )
    )

    #Uppdatera diagrammets utseende
    fig_folkmangd.update_layout(xaxis_title="Månad",
                                yaxis_title="Folkmängd",
                                template="plotly_white",
                                font_family=font_family_verdana)
    
    fig_folkmangd.update_traces(hovertemplate='<b>%{hovertext}</b><br>Månad: %{x}<br>Folkmängd: %{y: .0f}')
    
    return fig_folkmangd

#Kör appen
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)